## Erstmal auf den Zyklus gruppieren und dann Features erstellen

offen:
- Zyklen vorher - ges Zeit an entlade klaaen und warm + Anz Zyklen vorher
- Feature Auswahl
- nur Ref Discharge filtern
- in ein DF
- Zielvariable berechnen (current und RelTime als Features raus) 

### Import

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [43]:
def stark_leicht(type_, current_mean, relativeTime_amax):
    entladen_stark = 0
    entladen_leicht = 0
    laden_stark = 0
    laden_leicht = 0
    pause = 0
    
    
    if type_ == 'D': #discharge
        if current_mean >= 3:
            entladen_stark = relativeTime_amax
        else:
            entladen_leicht = relativeTime_amax
    if type_ == 'C': #charge
        if current_mean <= -3:
            laden_stark = relativeTime_amax
        else:
            laden_leicht = relativeTime_amax
    if type_ == 'R':
        pause = relativeTime_amax
    return entladen_stark, entladen_leicht, laden_stark, laden_leicht, pause

    

def temp_hoch(temperature_amax, relativeTime_amax):
    temp_hoch = 0
    if temperature_amax >= 30:
        return relativeTime_amax
    else:
        return 0
        

# for file in os.listdir(p_fold):
#     if ".csv" in file:
file = "prepared_RW1.csv"
logging.info(f"<<< Started processing file: {file} >>>")
new_filename = file.replace("prepared","processed")

df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                         #nrows= 100000
                        )
logging.info(f"<<< Size of Dataframe: {df.shape} >>>")

#Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
df_pv = pd.pivot_table(
   df,
   index=['batteryname','zyklus','comment','type'],
   aggfunc={'time': np.min, 'relativeTime':  np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
).reset_index()

df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]



df_pv['time_entladen_stark'],df_pv['time_entladen_leicht'],df_pv['time_laden_stark'],df_pv['time_laden_leicht'],df_pv['time_pause']  = zip(*df_pv.apply(lambda x: stark_leicht(x['type_'],x['current_mean'],x['relativeTime_amax']),axis=1))
#Lade

df_pv['time_temp_hoch'] = df_pv.apply(lambda x: temp_hoch(x['temperature_amax'],x['relativeTime_amax']),axis=1)


logging.info(f"<<< Start saving dataframe with shape {df_pv.shape} >>>")
df_pv.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[09:10:55] INFO: <<< Started processing file: prepared_RW1.csv >>>
[09:11:00] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[09:11:02] INFO: <<< Start saving dataframe with shape (17065, 21) >>>
[09:11:02] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>


In [41]:
stark_leicht("D",2,1)

(0, 1)

pause              9085
entladen leicht    5003
entladen stark     1696
laden leicht       1281
Name: ladearten, dtype: int64